# coverage_cycles_passes 

In [ ]:
dataset = ""
token = ""

In [ ]:
#!/usr/bin/env python3 
import os
import json
import requests
import pandas as pd
from tqdm import tqdm
from io import StringIO
from datetime import datetime
tqdm.pandas()

cmr: str = "cmr.earthdata.nasa.gov"

#token: str = os.environ.get('URS_TOKEN')

ShortName: str = f"SWOT_{dataset}_1.0"

def _search(params: dict, concept: str="granules", cmr: str=cmr, token: str=token, verbose: bool=True) -> list:
    url: str = f"https://{cmr}/search/{concept}.umm_json"
    headers: dict = {}
    if token is not None:
        headers['Authorization'] = f'Bearer {token}'
    results: list = []
    while True:
        with requests.get(url=url, params=params, headers=headers) as r:
            response = r.json()
            if 'errors' in response:
                raise Exception(response.get('errors'))
            for k,v in r.headers.items():
                if k.lower().startswith("cmr"):
                    headers.update({k.lower():v})
        items = response.get("items")
        if type(items) is list:
            if len(items)>0:
                results.extend(items)
        if verbose:
            print((f"[{datetime.utcnow().strftime('%H:%M:%S')}]"
                   f" {len(results)} / {response.get('hits')}"))
        if any([items is None, 
                len(results)==response.get('hits'), 
                len(results)==headers.get('hits')]):
            break
    return results

def search(provider: str="POCLOUD", page_size: int=2000, pattern: str=None, **kwargs):
    params: dict = {'provider':provider, 'page_size':page_size}
    if pattern is not None and pattern in kwargs:
        params[f'options[{pattern}][pattern]'] = 'true'
        params[f'{pattern}[]'] = kwargs.get(pattern)
        del kwargs[pattern]
    params.update(kwargs)
    collections = _search(params=params, concept="collections")
    granules= _search(params=params)
    return collections, granules

In [ ]:
results = search(ShortName=ShortName, pattern="ShortName")

In [ ]:
coll = pd.concat(pd.Series(results[0]).apply(pd.json_normalize).tolist()).set_index("umm.ShortName")

gran = pd.DataFrame(results[1])
gran['revision-id'] = gran.meta.apply(lambda x: x.get("revision-id"))
gran['concept-id'] = gran.meta.apply(lambda x: x.get("concept-id"))
gran['native-id'] = gran.meta.apply(lambda x: x.get("native-id"))
gran['GranuleUR'] = gran.umm.apply(lambda x: x.get("GranuleUR"))
gran['ShortName'] = gran.umm.apply(lambda x: x.get("CollectionReference") \
                                              .get("ShortName")).tolist()
gran['Cycle'] = gran.GranuleUR.apply(lambda x: int(x.split("_")[4]))
gran['Pass'] = gran.GranuleUR.apply(lambda x: int(x.split("_")[5]))
gran['Track'] = gran.umm.apply(lambda x: x.get('SpatialExtent') \
                                          .get('HorizontalSpatialDomain') \
                                          .get("Track")).tolist()

gran['HasTrack'] = (~gran.Track.isnull()).tolist()
gran.loc[gran['HasTrack'],'TrackCycle'] = gran[gran['HasTrack']].Track \
    .apply(lambda x: x['Cycle'])

gran['TrackCycle'] = gran.TrackCycle.fillna(0.0).astype(int)
gran.loc[gran['HasTrack'],'TrackPass'] = gran[gran['HasTrack']].Track \
    .apply(lambda x: x['Passes'][0].get("Pass"))

gran['TrackPass'] = gran.TrackPass.fillna(0.0).astype(int)
gran['TrackMatch'] = gran.apply(lambda x: [x.Cycle,x.Pass]==[x.TrackCycle,x.TrackPass], axis=1)

print("Cycle/pass metadata status:", gran.TrackMatch.sum(), "/", gran.index.size)

Print some granule records by their cycle and pass numbers, or not:

In [ ]:
cycles, passes = [407], [1]

print(json.dumps(gran[gran.Cycle.isin(cycles) & gran.Pass.isin(passes)].to_dict("records"), indent=2))

In [ ]:
gran['CRID'] = gran.apply(lambda x: f"{x.umm.get('GranuleUR').split('_')[-2]}", axis=1)
bycrid = gran.groupby(['Cycle','Pass']).CRID \
    .apply(lambda x: f"{sorted(x)[-1]}").to_frame().reset_index() \
    .pivot(index="Cycle", columns="Pass", values='CRID')

gran['Versions'] = gran.apply(lambda x: f"{x.umm.get('GranuleUR').split('_')[-1]}", axis=1)
byvers = gran.groupby(['Cycle','Pass']).Versions \
    .apply(lambda x: f"{x.count()}").to_frame().reset_index() \
    .pivot(index="Cycle", columns="Pass", values='Versions')

gran['String'] = gran.apply(lambda x: f"{x.umm.get('GranuleUR').split('_')[-2]}", axis=1)
tmp = gran.groupby(['Cycle','String']).GranuleUR.count().reset_index() \
    .pivot(index="Cycle", columns="String").fillna(0.0).astype(int)
tmp['Granules'] = tmp.index.to_series().apply(lambda x: gran[gran.Cycle==x].sort_values("Pass")).tolist()

cycles = sorted(tmp.index.tolist(), reverse=False)

def datefmt(x: str):
    return datetime.strptime(x, "%Y%m%dT%H%M%S").strftime("%-m/%-d %H:%M")

tstamps = tmp.apply(lambda x: (
        datefmt(x.Granules.iloc[0].GranuleUR.iloc[0].split("_")[6]), 
        datefmt(x.Granules.iloc[-1].GranuleUR.iloc[-1].split("_")[7]), ), axis=1) \
    .apply(lambda x: f"{x[0]}\n{x[1]}")

tmpbycrid = bycrid.sort_index(ascending=False).copy()

tmpbycrid['n / 28'] = (~tmpbycrid.isnull()).apply(lambda x: f"{(int(x.mean().round(2)*100))}%".rjust(4," "), axis=1)

tmpbyvers = byvers.sort_index(ascending=False).copy().reset_index(drop=True)

In [ ]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec
import matplotlib.font_manager as mfonts
from matplotlib import MatplotlibDeprecationWarning, cm, rc

warnings.filterwarnings("ignore", category=MatplotlibDeprecationWarning)

rc('text', usetex=False)

fig = plt.figure(figsize=(20, 64))

gs = gridspec.GridSpec(1, 2, width_ratios=[1.0, 3.75])

plt.autoscale(tight=True)

ax1 = plt.subplot(gs[0])

tmp.T.reset_index().T.iloc[1:].T.set_index("String").T.plot.barh(stacked=True, width=0.6, ax=ax1)

ax1.tick_params(top=True, labeltop=True, bottom=True, labelbottom=False, left=True, labelleft=True, right=False, labelright=False)
ax1.set_yticklabels(tstamps, fontsize=11, weight='semibold', )
plt.setp(ax1.get_xticklabels(), fontsize=11, weight="semibold")
ax1.axvline(x=28, color="black", alpha=0.75, )
ax1.set_frame_on(False)
ax1.set_ylabel(None)
ax1.set_xlabel(None)
ax1.set_xlim(0, 35)
ax1.grid(alpha=0.5)

plt.legend(title=False, facecolor="white", edgecolor="black", loc="upper right", framealpha=0.8, fontsize=11, shadow=True) 

ax2 = plt.subplot(gs[1])

ax2.axis('off')

tab = pd.plotting.table(ax=ax2, data=tmpbycrid, cellLoc='center', rowLoc='center', bbox=[0.000, 0.000, 1.000, 1.0067], )
tab.auto_set_font_size(False)
tab.set_fontsize(10.5)
tab.scale(1.0, 1.0)

max_vers = int(tmpbyvers.astype(float).max().max())

cmap_colors = dict(zip(list(range(2, max_vers+1)), ["yellow", "orange", "red"]))

if False:
    cmap_scale = cm.ScalarMappable(norm=plt.Normalize(vmin=2, vmax=max_vers-1), 
                                   cmap=mcolors.LinearSegmentedColormap.from_list("", cmap_colors))
    cmap_colors = {i: cmap_scale.to_rgba(i) for i in list(cmap_colors)}

for (row, col), cell in tab.get_celld().items():
    if row % 2 == 1:
        cell.set_facecolor("gray")
        cell.set_alpha(0.20)
    if (row == 0) or (col == -1) or (col == 28):
        cell.set_text_props(fontproperties=mfonts.FontProperties(weight='semibold', size=11))
        cell.set_text_props(color="white")
        cell.set_facecolor("black")
        cell.set_alpha(0.75)
    else:
        try:
            value = tmpbyvers.iloc[row-1, col]
        except IndexError as e:
            print("IndexError:", row, col)
        else:
            if str(value)=='nan':
                cell.set_facecolor("red")
                cell.set_alpha(0.5)
            elif int(value) in list(cmap_colors):
                cell.set_edgecolor(cmap_colors.get(int(value)))
                cell.set_linewidth(3.0)
            else:
                pass

plt.tight_layout()

plt.savefig(f'tests/{dataset}.png', bbox_inches='tight', transparent=False)